# 4 Failure Toxic Pathways

attempt to understand pathways that are too TOXIC to treat. to do that, we first select out all the drugs in ['arms']['interventions']['TRIAL'] or whatever. Take all the drugs that were in phase I trials that NEVER show up in phase II trials. To me that represents a set of EITHER 1) not enough accrural but more likely (given its very small numbers required for phase 1) it was too toxic in the dose escalation phase to continue


then you take all those drugs and look at the biomarkers required for inclusion in trials that contain them and perform the above.


then look at those drugs and see what they target (either biomarkers wise or with another data base)


In [2]:
from collections import defaultdict, Counter
import pandas as pd
import os, glob
import requests
import json
import re
from scipy import stats


In [85]:
def get_phase_study(study):
    """
    get phase of trial as a string, if multi-phase trial, trial gets upgraded to highest phase
    input:
    study <dict>
    return <str> phase
    """
    phase_str = str(study['phase']).strip()
    if phase_str=='I':
        return 'I'
    elif phase_str=='II' or phase_str =='I_II':
        return 'II'
    elif phase_str=='III' or phase_str =='II_III':
        return 'III'
    elif phase_str=='IV':
        return 'IV'
    elif phase_str=='O':
        return 'other'
    elif phase_str=='NA':
        return 'other'
    else:
        print(phase_str, type(phase_str), phase_str=='0' )
        return 'other'


def load_trial_data (json_trial_file):
	#interventional_trials_with_descendants2024-08-23.json

	active_trials = json.load(open(json_trial_file,'r'))
	active_trials = active_trials['data']

	nciid_to_trial = {}
	for trial in active_trials:
		nciid_to_trial[trial['nci_id']] = trial

	return nciid_to_trial    

def get_studystop_mapper(reason_excel_file):
	# input: studystop_reason_df.xlsx
	# output: dict: for reason to group of reason trial stopped (str)

	studystop_dict_filt_df_annon  = pd.read_excel(reason_excel_file)

	#cleaning some stuff
	studystop_dict_filt_df_annon['reason'] = studystop_dict_filt_df_annon['reason'].str.replace("_x000D_", "\r")
	studystop_dict_filt_df_annon['reason'] = studystop_dict_filt_df_annon['reason'].str.replace('\W', '')
	studystop_dict_filt_df_annon.reason.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
	studystop_dict_filt_df_annon.reason.replace({r'[^\w\s]':''}, regex=True, inplace=True)

	reason_to_group_mapper = pd.Series(studystop_dict_filt_df_annon.group.values, index=studystop_dict_filt_df_annon.reason.values).to_dict()

	return reason_to_group_mapper


def get_reason_stop(nci_id, nciid_to_trial, reason_to_group_mapper):
	
	trial  = nciid_to_trial[nci_id]

	str_reason = trial['why_study_stopped']
	
	if str_reason is not None:
		#process reason
	    str_reason = str_reason.lower()
	    str_reason = re.sub(r'[^\w\s]','',str_reason)
	    str_reason = re.sub(r'[^\x00-\x7F]','',str_reason)
	        

	    if str_reason not in reason_to_group_mapper:
	        reason_group = 'other'
	    else:
	        reason_group = reason_to_group_mapper[str_reason]

	else:
		reason_group = 'continued_trial'

	return reason_group

def get_biomarkers(trial):
    genes = [g for g in trial['biomarkers_new']['inclusion']['TREE']['symbols_dz'] if type(g)==str]
    return genes

def get_drug_info(trial):
    interv_drugs_dict = defaultdict(set)
    for arm in trial['arms']:
        for interv in arm['interventions']:        
            if interv['category']=='agent': # for interv['type'] Biological/Vaccine or Drug
                interv_drugs_dict[interv['name']].add(interv['nci_thesaurus_concept_id'])
    return interv_drugs_dict


In [5]:
%%time
active_trials = json.load(open('interventional_trials_with_descendants2024-08-23.json','r'))
print(len(active_trials['data']), active_trials['total'])
active_trials = active_trials['data']

20894 20894


In [4]:
%%time

nciid_to_trial = load_trial_data ('interventional_trials_with_descendants2024-08-23.json')
reason_to_group_mapper = get_studystop_mapper('studystop_reason_df.xlsx')
 


get drugs in all clinical trials split by those that where phase 1 
and those that with phase 2, 3, or 4
remove those that are phase other

In [57]:
drugs_all = defaultdict(set) # this a mapper between drug name <str> and NCI_Ids <set>
drugs_phase1 = defaultdict(set)
drugs_phaselater = defaultdict(set)
drug_to_nci_id = defaultdict(list)

for trial in active_trials:
    nci_id = trial['nci_id']
    phase = get_phase_study(trial)
    drug_dict = get_drug_info(trial)
    for drug, nci_set in drug_dict.items():
        drugs_all[drug] = drugs_all[drug].union(nci_set)
        drug_to_nci_id[drug].append(nci_id)
        if phase == 'I':
            drugs_phase1[drug] = drugs_all[drug].union(nci_set)
        elif (phase == 'II') or (phase == 'III') or (phase == 'IV'):
            drugs_phaselater[drug] = drugs_all[drug].union(nci_set)
    

In [58]:
len(drugs_all), len(drugs_phase1), len(drugs_phaselater)

(4915, 2939, 3159)

In [61]:
# for drug, nci_set in drugs_all.items():
#     if len(nci_set)>1:
#         print(drug, nci_set)

this shows that there is NOT a 1:1 mapping between drug and nci thesaurus id

In [64]:
#find drugs that drugs_phase1 but NOT in drugs_phaselater
drugs_phase1_only = {}
for drug, nci_set in drugs_phase1.items():
    if drug not in drugs_phaselater.keys():
        drugs_phase1_only[drug] = drugs_all[drug]
len(drugs_phase1_only)

1628

there are 1628 drugs/biologics that are only in phase 1 and not in phase II +

now find the biomarkers of these trials

In [99]:

# Pegylated CD25/CD122-selective Interleukin-2 Mutein STK-012

In [100]:
phase1_trials_only = set()
phase1_biomarkers = set()
no_biomarkers_arr = set()
for drug in drugs_phase1_only.keys():
    drug_trials = drug_to_nci_id[drug]
    for trial_id in drug_trials:
        phase1_trials_only.add(trial_id)
        trial_data = nciid_to_trial[trial_id]
        biomarkers = get_biomarkers(trial_data)
        if len(biomarkers) ==0:
            no_biomarkers_arr.add(drug)
        phase1_biomarkers = phase1_biomarkers.union(set(biomarkers))
print(len(phase1_trials_only), len(phase1_biomarkers))


phaselater_trials = set()
phaselater_biomarkers = set()
for drug in drugs_phaselater.keys():
    drug_trials = drug_to_nci_id[drug]
    for trial_id in drug_trials:
        phaselater_trials.add(trial_id)
        trial_data = nciid_to_trial[trial_id]
        biomarkers = get_biomarkers(trial_data)
        phaselater_biomarkers = phaselater_biomarkers.union(set(biomarkers))
print(len(phaselater_trials), len(phaselater_biomarkers))
        # reason_stop = get_reason_stop(trial_id, nciid_to_trial, reason_to_group_mapper)
        # if reason_stop !='continued_trial':
        #     print(drug, drugs_all[drug],biomarkers)

1897 236
15065 551


In [98]:
print(len(phase1_biomarkers))
print(len(phaselater_biomarkers))
print(len(phase1_biomarkers - phaselater_biomarkers))
print(sorted(phase1_biomarkers - phaselater_biomarkers))
# print(sorted(phaselater_biomarkers - phase1_biomarkers))

236
551
19
['CAMTA1', 'CD37', 'CDH3', 'GRN', 'GRPR', 'HHLA2', 'KMT2C', 'LYPD3', 'MAGEC1', 'MUC17', 'NSD2', 'NTSR1', 'PDE3A', 'PRLR', 'PTK7', 'SLFN12', 'TCIRG1', 'WWTR1', 'YAP1']


In [102]:
print(len(no_biomarkers_arr))# 

1073


In [106]:
drug_to_nci_id['Docirbrutinib']


['NCI-2023-06023']

In [115]:
# nciid_to_trial['NCI-2023-06023']['why_study_stopped']

In [120]:
drugs_phase1['Docirbrutinib']

{'C193467'}

In [103]:
no_biomarkers_arr

{'Anti-PD-L1 Antibody/IL-15 Fusion Protein IGM-7354',
 'Anti-SIRPa Monoclonal Antibody ADU-1805',
 'Expanded/Activated Gamma Delta T-cells',
 'CD200 Activation Receptor Ligand hP1A8',
 'DNA Plasmid Vaccine ITI-1001',
 'Dimethyl Fumarate',
 'BET Inhibitor ABBV-744',
 'Saponin-Cholesterol-Phospholipid Adjuvant',
 'DNA-dependent Protein Kinase Inhibitor VX-984',
 'Glutaminase-1 Inhibitor IACS-6274',
 'Tasadenoturev-infected Allogeneic Bone Marrow-derived Mesenchymal Stem Cells',
 'Glioma Lysate Vaccine GBM6-AD',
 'Warfarin',
 'Actinium Ac 225-DOTA-h11B6 JNJ-69086420',
 'Maraba Oncolytic Virus Expressing Mutant HPV E6/E7',
 'Conditionally-activated Anti-EpCAM Antibody-drug Conjugate CX-2051',
 'Sotrovimab',
 'Cirtuvivint',
 'Autologous Anti-GFRa4 CAR-TCR-zeta-4-1BB-expressing T-cells',
 'Bcl-2 Inhibitor LOXO-338',
 'Docirbrutinib',
 'PD-1 Targeted IL-15 Mutein Fusion Protein PF-07209960',
 'IDO1 Inhibitor KHK2455',
 'Anti-CD38 Monoclonal Antibody SAR442085',
 'Anti-5T4 Antibody-drug Conjug

In [121]:
import requests

def get_nci_target(nci_code):
    url = f"https://services.nci.nih.gov/syndication/v1/ncit/{nci_code}/relationships"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad responses
        data = response.json()
        
        # Search for the 'Has_Target' relationship
        for relationship in data.get('relationships', []):
            if relationship.get('relationshipType', {}).get('name') == 'Has_Target':
                return relationship.get('target', {}).get('name')
        
        return "No target found for this NCI code."
    
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Example usage
nci_code = 'C193467'
target_name = get_nci_target(nci_code)
print(f"NCI Code: {nci_code} has target: {target_name}")


NCI Code: C193467 has target: An error occurred: HTTPSConnectionPool(host='services.nci.nih.gov', port=443): Max retries exceeded with url: /syndication/v1/ncit/C193467/relationships (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x5268a2550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))


1897 of the total 5156 phase 1 trials have drugs only studied in phase I. These trials have a total of 236 biomarkers noted as inclusion criteria within the nci api. However, only 19 of those ['CAMTA1', 'CD37', 'CDH3', 'GRN', 'GRPR', 'HHLA2', 'KMT2C', 'LYPD3', 'MAGEC1', 'MUC17', 'NSD2', 'NTSR1', 'PDE3A', 'PRLR', 'PTK7', 'SLFN12', 'TCIRG1', 'WWTR1', 'YAP1'] are biomarkers specific to phase 1 trials 

There are 1073 drugs without biomarkers associated with them that are stored in the nci api database. 




In [74]:
nci_ids = drug_to_nci_id['Pegylated CD25/CD122-selective Interleukin-2 Mutein STK-012']
nci_ids

['NCI-2022-00633']

In [76]:
test = nciid_to_trial['NCI-2022-00633']
get_biomarkers(test)

[]

In [78]:
test.keys()

dict_keys(['other_ids', 'amendment_date', 'keywords', 'dcp_id', 'interventional_model', 'lead_org', 'eligibility', 'sites', 'completion_date_type_code', 'detail_description', 'official_title', '_phase_sort_order', 'collaborators', 'associated_studies', 'outcome_measures', 'phase', 'central_contact', 'primary_purpose', 'number_of_arms', '_study_protocol_type_sort_order', 'nct_id', 'biomarkers', 'classification_code', 'current_trial_status_date', 'diseases', '_primary_purpose_sort_order', 'protocol_id', 'active_sites_count', 'lead_org_cancer_center', 'arms', 'study_model_code', 'nci_id', 'why_study_stopped', 'brief_summary', 'brief_title', 'status_history', 'study_population_description', 'sampling_method_code', 'minimum_target_accrual_number', 'prior_therapy', '_current_trial_status_sort_order', 'start_date', 'record_verification_date', 'ctep_id', 'current_trial_status', 'study_model_other_text', 'masking', 'acronym', 'nci_programs', 'nci_funded', 'anatomic_sites', 'ccr_id', 'start_date

In [80]:
test['why_study_stopped']